In [1]:
import torch
import numpy as np
import numpy as np
import time
import itertools
import random
import networkx as nx
import collections
import generation
from utils import loop_counter

c:\Users\uitak\anaconda3\envs\ldm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
unit=3
Batch_Size  = 1
In_Channel  = 1
Height      = unit+1
Width       = unit+1
Depth       = unit+1
Out_Channel = 1
Group       = 1
Kernel_Size = 3
Kernel_Size_D = Kernel_Size
Kernel_Size_H = Kernel_Size
Kernel_Size_W = Kernel_Size
Padding     = 1
Dilation    = 1

thr = torch.ones(Batch_Size,In_Channel,Depth,Height,Width)
filter = torch.ones(Out_Channel,int(In_Channel/Group),Kernel_Size_D,Kernel_Size_H,Kernel_Size_W)
filter[0][0][1][1][1]=0
result = torch.conv3d(thr,filter,padding=Padding)

In [4]:
from simanneal import Annealer
import random
import itertools

class LifeGameProblem(Annealer):

    def __init__(self, init_state):
        super(LifeGameProblem, self).__init__(init_state)  # important!

    # 探索点の移動ルール
    def move(self):
        mi = Width*Height*Depth//10
        for i in range(mi):
            j = random.randint(0,Width*Height*Depth-1)
            if self.state[0][0][j // (Height * Width)][(j % (Height * Width)) // Width][j % Width]==0:
                self.state[0][0][j // (Height * Width)][(j % (Height * Width)) // Width][j % Width]=1
            else:
                self.state[0][0][j // (Height * Width)][(j % (Height * Width)) // Width][j % Width]=0


    # 目的関数
    def energy(self):
        thr = torch.from_numpy(self.state)
        r = generation.generation(thr,filter,100)
        m = []
        for _, g in itertools.groupby(r[1]):
            m.append(len(list(g)))
        return -2*r[0]+torch.sum(thr)*0.3+max(m)**2-loop_counter.loop_counter(r[1])**3



# 初期割当て
init_state = np.zeros((Batch_Size, In_Channel, Depth, Height, Width), dtype=np.float32)
i = random.randint(0,Width*Height*Depth-1)
for j in range(Depth * Height * Width):
        if ((i >> j) & 1):
            init_state[0][0][j // (Height * Width)][(j % (Height * Width)) // Width][j % Width] = 1

prob = LifeGameProblem(init_state)
prob.steps = 10000
prob.Tmax = 500000
prob.copy_strategy = "slice"
prob.anneal()  # 焼きなまし実行

for i,s in enumerate(prob.state):
    print(generation.generation(torch.from_numpy(s),filter,500))


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
  6174.46319       7552.30    96.00%    47.00%     0:02:09     0:03:50

[500, [5451484206851430657, 7757328874694252801, 7748320612871313825, 8903495017927743905, 8903475776440736167, 8903475776977484167, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 8903475775903742343, 890347